# kaggle score: 0.5502 R^2 - Gradient Boost Model (Normalized frequency encoding of categorical features)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt       
%matplotlib inline 
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, RandomizedSearchCV
import json
from scipy.stats import randint as sp_randint

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/merc/train.csv")
train

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/merc/test.csv")
test

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
y = train["y"]
train = train.drop(["ID","y"], axis=1) 

In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/merc/rf_features.txt", "r") as fp:
  rf_binfeats = json.load(fp)

with open("/content/drive/My Drive/Colab Notebooks/merc/gb_features.txt", "r") as fp:
  gb_binfeats = json.load(fp)

In [ ]:
catcols = train.iloc[:,:8].columns.to_list()
catcols.remove("X4")
catcols

['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8']

**Combining categorical and binary features (for respective models) that we have selected in part 1:**

In [ ]:
xtr_rf = train[rf_binfeats+catcols]
xtr_gb = train[gb_binfeats+catcols]

xte_rf = test[rf_binfeats+catcols]
xte_gb = test[gb_binfeats+catcols]

print("train and test sets for randomforest:")
print(xtr_rf.shape)
print(xte_rf.shape)

print("train and test sets for gradientboosting:")
print(xtr_gb.shape)
print(xte_gb.shape)

train and test sets for randomforest:
(4209, 25)
(4209, 25)
train and test sets for gradientboosting:
(4209, 17)
(4209, 17)


**Normalized frequency encoding. As the name suggests each label will be replaced by its normalized frequency:**

In [ ]:
for col in catcols:
    xtr_rf[col+"_new"] = xtr_rf[col]
    xtr_rf[col+"_new"].replace(xtr_rf[col].value_counts(normalize=True).to_dict(),inplace=True)
    xtr_gb[col+"_new"] = xtr_gb[col]
    xtr_gb[col+"_new"].replace(xtr_gb[col].value_counts(normalize=True).to_dict(),inplace=True)
    
    xte_rf[col+"_new"] = xte_rf[col]
    xte_rf[col+"_new"].replace(xtr_rf[col].value_counts(normalize=True).to_dict(),inplace=True)
    xte_gb[col+"_new"] = xte_gb[col]
    xte_gb[col+"_new"].replace(xtr_gb[col].value_counts(normalize=True).to_dict(),inplace=True)


xtr_rf.drop(catcols,axis=1,inplace=True)
xte_rf.drop(catcols,axis=1,inplace=True)

xtr_gb.drop(catcols,axis=1,inplace=True)
xte_gb.drop(catcols,axis=1,inplace=True)

**We know that test set has labels that are not present in train set. Hence they wont be encoded. We will replace those labels with 0's:**

In [ ]:
for col in catcols:
    xte_rf[col+"_new"] = xte_rf[col+"_new"].apply(lambda x: np.nan if type(x)==str else x)
    xte_gb[col+"_new"] = xte_rf[col+"_new"].apply(lambda x: np.nan if type(x)==str else x)

xte_rf.fillna(0,inplace=True)
xte_gb.fillna(0,inplace=True)

**Defining some functions which we will be using:**

In [ ]:
def training_and_tuning(model,xtr,ytr,parameters,iterations):
  print("started random search cv:\n")
  randomCV = RandomizedSearchCV(model, param_distributions=parameters, n_iter=iterations, cv=5, random_state=7)
  randomCV.fit(xtr, ytr)
  best = randomCV.best_params_
  best_sc = randomCV.best_score_
  print("best score:")
  print(best_sc)
  print("best parameters obtained:")
  print(best)
  return best

In [ ]:
def tuned_model(cl,xtr,ytr):
  scores = cross_val_score(cl, xtr, ytr, cv=5)
  print("5 cv score: {}".format(scores.mean()))

In [ ]:
def kaggle_testset(final_model,trainset,trainlabels,testset,filename):
  final_model.fit(trainset,trainlabels)
  final_predictions = final_model.predict(testset)

  submit = pd.DataFrame(columns=["ID","y"])
  submit["y"] = final_predictions
  submit["ID"] = test["ID"]
  submit.to_csv("/content/drive/My Drive/Colab Notebooks/merc/"+filename,index=False)

# randomforest:

In [ ]:
%%time
model = RandomForestRegressor(random_state=7)

parameters = {"max_features": list(np.arange(1,25))+[None,"sqrt"],
              "max_depth":list(np.arange(3,20))+[None],
              "n_estimators": np.arange(10,150,5)}

best = training_and_tuning(model,xtr_rf,y,parameters,100)

started random search cv:

best score:
0.5666609143213326
best parameters obtained:
{'n_estimators': 10, 'max_features': 24, 'max_depth': 4}
CPU times: user 4min 4s, sys: 533 ms, total: 4min 5s
Wall time: 4min 5s


In [ ]:
cl = RandomForestRegressor(n_estimators=10, max_features=24, max_depth=4, random_state=7)
tuned_model(cl, xtr_rf, y)

5 cv score: 0.5666609143213326


In [ ]:
kaggle_testset(cl, xtr_rf, y, xte_rf, "rf_submission2.csv")

**kaggle score: 0.5424 (private leaderboard)**
# GradientBoost:

In [ ]:
%%time
model = GradientBoostingRegressor(random_state=7)

parameters = {"max_features": list(np.arange(1,17))+[None,"sqrt"],
              "max_depth":list(np.arange(3,20))+[None],
              "n_estimators": np.arange(10,150,5)}

best = training_and_tuning(model,xtr_gb,y,parameters,100)

started random search cv:

best score:
0.5712443762273305
best parameters obtained:
{'n_estimators': 65, 'max_features': 7, 'max_depth': 3}
CPU times: user 4min 47s, sys: 151 ms, total: 4min 48s
Wall time: 4min 48s


In [ ]:
cl = GradientBoostingRegressor(n_estimators=65, max_features=7, max_depth=3, random_state=7)
tuned_model(cl, xtr_gb, y)

5 cv score: 0.5712443762273305


In [ ]:
kaggle_testset(cl, xtr_gb, y, xte_gb, "gb_submission2.csv")

**kaggle score: 0.5502 (private leaderboard)**